kesin doğru sezgi: günümüzde **autoencoder (AE) tabanlı yaklaşımlar**, RBM/DBN’lere göre hem pratikte hem performansta çoğu senaryoda önde. “iyice öğreneyim” diyorsan, konuyu **3 farklı veri tipi** üzerinde denemen en öğreticisi olur: görüntü (denoising), tabular (anomali), etkileşim matrisi (öneri). Aşağıda **adım adım egzersiz rotası**, uygun **veri setleri**, **başarı metrikleri** ve her birine mini “başlangıç kod iskeleti” var. İster sırayla git, ister birini seçip oradan başlayalım.

---

# 0) Hızlı rota (özet)

1. **Denoising Autoencoder (DAE) – Görüntü**

   * Veri: **Fashion-MNIST** → gürültü ekle → gürültüyü kaldır
   * Amaç: Conv AE, Skip-connection (U-Net mini), MSE / SSIM ile ölç
2. **Anomali Tespiti – Tabular**

   * Veri: **Credit Card Fraud (ULB)** veya **NSL-KDD**
   * Amaç: AE’nin **rekonstrüksiyon hatası** → anomali skoru (ROC-AUC, PR-AUC)
3. **Öneri – Kullanıcı×Öğe**

   * Veri: **MovieLens (ml-100k veya ml-1m)**
   * Amaç: **User/Item Autoencoder** (veya VAE) ile top-K öneri (Precision\@K, Recall\@K, NDCG)

---

# 1) Denoising Autoencoder (Fashion-MNIST)

## Neden bu egzersiz?

* Konvolüsyonel AE mimarisini, **encoder–bottleneck–decoder** akışını çok hızlı görürsün.
* Gürültü (Gaussian / salt-pepper) ekleyip tekrar temizlemek **AE’nin sezgisel gücünü** gösterir.
* Değerlendirme: **MSE**, **PSNR**, **SSIM** + görsel kıyas.

## Veri

* **Fashion-MNIST** (60k train, 10k test, 28×28 gri); Keras `tf.keras.datasets.fashion_mnist` veya Kaggle.

## Minimal başlangıç kod iskeleti (Keras)

```python
import tensorflow as tf
from tensorflow.keras import layers, models

# 1) Data
(x_train, _), (x_test, _) = tf.keras.datasets.fashion_mnist.load_data()
x_train = (x_train.astype("float32")/255.0)[..., None]  # (N,28,28,1)
x_test  = (x_test.astype("float32")/255.0)[..., None]

# Gürültü ekle (denoising)
noise = 0.4
x_train_noisy = tf.clip_by_value(x_train + noise*tf.random.normal(tf.shape(x_train)), 0., 1.)
x_test_noisy  = tf.clip_by_value(x_test + noise*tf.random.normal(tf.shape(x_test)), 0., 1.)

# 2) Model (Conv AE)
def build_dae():
    inp = layers.Input((28,28,1))
    x = layers.Conv2D(32,3,activation="relu",padding="same")(inp)
    x = layers.MaxPool2D()(x)                  # 14x14
    x = layers.Conv2D(64,3,activation="relu",padding="same")(x)
    z = layers.MaxPool2D()(x)                  # 7x7 (bottleneck)

    x = layers.Conv2DTranspose(64,3,strides=2,activation="relu",padding="same")(z)  # 14x14
    x = layers.Conv2DTranspose(32,3,strides=2,activation="relu",padding="same")(x)  # 28x28
    out = layers.Conv2D(1,3,activation="sigmoid",padding="same")(x)
    return models.Model(inp, out)

dae = build_dae()
dae.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss="mse")
dae.fit(x_train_noisy, x_train, validation_split=0.1, epochs=10, batch_size=128)

# 3) Tahmin ve kalite ölçümü
denoised = dae.predict(x_test_noisy)
mse = tf.reduce_mean(tf.square(x_test - denoised)).numpy()
print("MSE:", mse)
```

> Geliştirme: **Skip bağlantılar** (U-Net mini), **SSIM** ölçümü, **Noise2Noise** veya **β-VAE** ile farklı latent yapı.

---

# 2) Autoencoder ile Anomali Tespiti (Tabular)

## Mantık

* AE **normal desenleri iyi öğrenir**, anormal olanları **kötü rekonstrükte eder** → **rekonstrüksiyon hatası** (örn. MAE/MSE) **anomali skoru** olur.

## Veri seçenekleri

* **Credit Card Fraud (ULB, Kaggle)**: Dengesiz + sayısal; **Standartizasyon** şart.
* **NSL-KDD**: Ağ trafiği; one-hot + scale; farklı atak sınıfları.
* (Alternatif) **UNSW-NB15**, **Arrhythmia (UCI)**.

## Minimal başlangıç (Keras, dense AE)

```python
import numpy as np, pandas as pd, tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, average_precision_score, precision_recall_curve

# 1) Data
df = pd.read_csv("creditcard.csv")   # ['Time','V1'..'V28','Amount','Class']
y = df["Class"].values.astype(int)
X = df.drop(columns=["Class"]).values

scaler = StandardScaler().fit(X)
X = scaler.transform(X)

# AE’yi normal sınıfta eğitmek daha tipik (unsupervised outlier)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_train_norm = X_train[y_train==0]   # sadece normal ile eğit
X_val = X_test                       # skor için kullan

# 2) Model
inp = tf.keras.Input(shape=(X.shape[1],))
h   = tf.keras.layers.Dense(32, activation="relu")(inp)
z   = tf.keras.layers.Dense(16, activation="relu")(h)   # bottleneck
h2  = tf.keras.layers.Dense(32, activation="relu")(z)
out = tf.keras.layers.Dense(X.shape[1], activation=None)(h2)

ae = tf.keras.Model(inp, out)
ae.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss="mse")
ae.fit(X_train_norm, X_train_norm, validation_split=0.1, epochs=50, batch_size=256, verbose=0)

# 3) Skor = rekonstrüksiyon hatası
X_hat = ae.predict(X_val, verbose=0)
err   = np.mean((X_val - X_hat)**2, axis=1)

print("ROC-AUC:", roc_auc_score(y_test, err))
print("PR-AUC :", average_precision_score(y_test, err))

# Eşik seçimi (ör. F1-opt)
prec, rec, thr = precision_recall_curve(y_test, err)
f1 = 2*prec[:-1]*rec[:-1]/(prec[:-1]+rec[:-1]+1e-12)
t_star = thr[f1.argmax()]
y_pred = (err >= t_star).astype(int)
```

> Geliştirme: **Sadece normalle eğit**, **Dropout/L1/L2** ile “sıkıştırmayı” zorlaştır, **RobustScaler**, **β-VAE** (latent düzenlilik), **Isolation Forest/LOF** ile kıyas.

---

# 3) Autoencoder ile Öneri (MovieLens)

## Mantık (Implicit feedback)

* Her kullanıcıyı **öğe vektörüne** (0/1/puan) map’le; **User-AE** ile sıkıştır → tekrar aç → **çıktı = p(“sever”)** gibi skor.
* Alternatif: **VAE-CF** (Mult-VAE) – pratikte çok güçlü.

## Veri

* **MovieLens ml-100k / ml-1m** (başlamak için 100k iyi).
* Giriş: kullanıcı satırı (N\_items uzunluğunda), değerler 0/1 (implicit); train/test split kullanıcı bazında.

## Minimal User-AE iskeleti

```python
import numpy as np, pandas as pd, tensorflow as tf
from sklearn.metrics import roc_auc_score

# 1) User×Item matrisini oluştur (ml-100k: u1.base/u1.test mantığı)
# ... (RBM’de yaptığın gibi) train_mat/test_mat üret
# Binarize: rating>=3 -> 1, rating 1-2 -> 0, bilinmeyen -> 0 (implicit varsayımı)

train_mat = ...  # (U, I) float32
test_mat  = ...

# 2) Model
I = train_mat.shape[1]
inp = tf.keras.Input(shape=(I,))
x   = tf.keras.layers.Dense(256, activation="relu")(inp)
x   = tf.keras.layers.Dropout(0.3)(x)
z   = tf.keras.layers.Dense(64, activation="relu")(x)     # bottleneck
x   = tf.keras.layers.Dense(256, activation="relu")(z)
out = tf.keras.layers.Dense(I, activation="sigmoid")(x)   # her item için p(like)

model = tf.keras.Model(inp, out)
# Not: İsteğe bağlı ağırlıklandırma (pozitiflere daha çok ağırlık) için custom loss yazılabilir.
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss="binary_crossentropy")

model.fit(train_mat, train_mat, validation_split=0.1, epochs=30, batch_size=256, verbose=1)

# 3) Öneri skoru ve top-K
pred = model.predict(train_mat, verbose=0)  # (U, I)
# zaten sevilenleri maskeden düş
pred_masked = pred.copy()
pred_masked[train_mat>0.5] = -1e9

def precision_recall_at_k(pred_masked, test_mat, K=10):
    P, R = [], []
    for u in range(pred_masked.shape[0]):
        topk = np.argsort(pred_masked[u])[-K:][::-1]
        pos  = np.where(test_mat[u] > 0.5)[0]
        if len(pos)==0: 
            continue
        hit = len(set(topk) & set(pos))
        P.append(hit/K); R.append(hit/len(pos))
    return float(np.mean(P)), float(np.mean(R))

P10, R10 = precision_recall_at_k(pred_masked, test_mat, K=10)
print("P@10:", P10, "R@10:", R10)
```

> Geliştirme: **VAE-CF (Mult-VAE)**, **item-side AE** (içerik özellikleri), **sampled softmax / BPR** benzeri sıralama kayıpları, **alpha-weighted BCE** (pozitiflere ağırlık).

---

# Ne beklemeliyim? (hedef metrikler)

* **DAE (Fashion-MNIST)**: Eğitim ilerledikçe MSE ↓, görseller gözle görülür şekilde temizlenir.
* **AE-Anomali (CreditCard)**: ROC-AUC **0.8+** (iyi), PR-AUC **pozitif oranından** belirgin yüksek.
* **AE-Recommender (ML-100k)**: `P@10 ~ 0.10–0.15` başlangıçta normal; mimari/regularization/negatif yönetimi ile ↑.

---

# Son olarak: hangi sırayla?

1. **Fashion-MNIST DAE** (ısınma) → 20–30 dk
2. **Credit Card AE-Anomali** (tabular sezgi) → 30–60 dk
3. **MovieLens User-AE** (öneri) → 60+ dk

   * beğenirsen **VAE-CF**’e terfi ederiz.

Hangisiyle başlayalım? İstersen **Fashion-MNIST DAE**’yi birlikte açalım; yukarıdaki kod direkt çalışır.
